In [ ]:
# PYradiomics

import os
import radiomics
import radiomics.featureextractor as FEE
import SimpleITK as sitk

# 
cn_folder = "/home/CN"
ep_folder = "/home//EP"
roi_files = [
    "/home/right_posterior_thalamic_radiation_roi.nii.gz",
    "/home/right_pallidum_roi.nii.gz"
]
output_csv = '/home/radiomics_features.csv'
parameter_file = '/home/exampleMR_3mm.yaml'

# 
extractor = FEE.RadiomicsFeatureExtractor(parameter_file)

# 
with open(output_csv, 'w') as csv_fp:
    flag = True
    
    # CN&EP
    for main_folder in [cn_folder, ep_folder]:
        for subject in os.listdir(main_folder):
            subject_path = os.path.join(main_folder, subject)
            if not os.path.isdir(subject_path):
                continue
            
            # T1WI
            t1wi_folder = os.path.join(subject_path, 'T1WI')
            t1wi_files = [f for f in os.listdir(t1wi_folder) if f.endswith('.nii')]
            if len(t1wi_files) == 0:
                print(f"No T1WI file found for subject {subject}")
                continue
            
            t1wi_file = os.path.join(t1wi_folder, t1wi_files[0])
            
            # ROI
            for roi_file in roi_files:
                try:
                    result = extractor.execute(t1wi_file, roi_file)
                    
                    # CSV
                    if flag:
                        csv_fp.write('subject,roi')
                        for k in result.keys():
                            if str(k).split("_")[0] != 'diagnostics':
                                csv_fp.write(',' + str(k))
                        csv_fp.write('\n')
                        flag = False

                    csv_fp.write(f"{subject},{os.path.basename(roi_file)}")
                    for k, v in result.items():
                        if str(k).split("_")[0] != 'diagnostics':
                            csv_fp.write(',' + str(v))
                    csv_fp.write('\n')
                    csv_fp.flush()

                except ValueError as e:
                    print(f"Error processing subject: {subject}, ROI: {os.path.basename(roi_file)}")
                    print(f"T1WI file: {t1wi_file}")
                    print(f"ROI file: {roi_file}")
                    print("Error:", e)
                    continue  

            print(f"Processed subject: {subject}")

print("Radiomics feature extraction complete.")
